# Tutorial: Convexity and Regularization

CPSC 340: Machine Learning and Data Mining

The University of British Columbia

Adapted from Mike Gelbart's notebooks

In [ ]:
import numpy as np
import numpy.random as npr
from sklearn.linear_model import LinearRegression, HuberRegressor, Ridge, Lasso
from scipy.optimize import minimize
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
matplotlib.rcParams['font.size'] = 16

Topics:

- Convexity
- Why L1 leads to sparsity
- Regularization strength intuition, regularization paths, L1 vs. L2
- Why not use Huber regularization?
- Uniqueness of solution: L1 and L2

## Convexity

Convex functions have many nice properties. For our purposes, the main one we care about is that all stationary points are global minima, i.e. $\nabla f(w) = 0$ implies $w$ is a global minimizer. 

How do we show that a function is convex? Firstly, its domain must be a convex set (which is not of concern to us in 340 since we are typically doing unconstrained optimization). Past that, there are three main approaches:

- Using the definition of convexity: $$f(\theta w + (1-\theta)v) \leq \theta f(w) + (1-\theta) f(v), \quad\forall w, v \in \mathbb{R}, \quad\forall \theta \in [0,1]$$This always applies.
- If $f$ is twice differentiable everywhere:
    - $f''(w) \geq 0$ for all $w$ (1D case)
    - $\nabla^2f(w) \succeq 0$ for all $w$. In words: all eigenvalue of the Hessian matrix are nonnegative. 
- Operations that preserve convexity

Commonly used operations that preserve convexity:
- Any p-norm is convex (for p >= 1)
- If $f$ and $g$ are convex functions:
    - $h(w) = \max(f(w), g(w))$ is convex
    - $h(w) = f(Aw+b)$ is convex for any $A, b$
    - $h(w) = \alpha f(w)$ is convex for $\alpha \geq 0$
    - $h(w) = f(w) + g(w)$ is convex 
- Note: f(g(w)) is NOT necessarily convex
- See lecture slides 

#### Examples
Show the following functions are convex
- $f(w) = -\log(w^2)$ on $0 < w < \infty$
- $f(w) = \frac12 \|Xw-y\|^2+\frac\lambda2\|w\|^2$ 

## Why L1 regularization leads to sparsity

Consider this 1D case of a feature whose "true" weight is close to zero. Starting with no regularization:

$$f(w)=\sum_{i=1}^n (wx_i-y_i)^2$$

In [ ]:
npr.seed(0)

# Generate random 1D data set
d = 1
n = 30
X = npr.randn(n,d)
y = npr.randn(n,1)
def f(w, X, y):
    return np.sum((X@w-y)**2,axis=0)

wgrid = np.arange(-1,1,2**(-5))
fw = f(wgrid[None], X, y)
minind = np.argmin(fw);
minw = wgrid[minind]
plt.plot(wgrid, fw);
plt.xlabel("$w$");
plt.ylabel("$f(w)$");
plt.plot((minw, minw),plt.ylim(),'--k');
plt.title("No regularization"); # $f(w)=\sum_{i=1}^n (wx_i-y_i)^2$
print("minimizer: w=%.2f" % minw);

Before proceeding, ask yourself: do you understand what this picture is showing? It's the parameter space, but in 1D, so we don't need a contour plot like we did in 2D.

#### L0 regularization

$$f(w)=\sum_{i=1}^n (wx_i-y_i)^2 + \lambda \|w\|_0$$

In [ ]:
def f0(w, X, y, λ=10):
    return np.sum((X@w-y)**2,axis=0) + λ*(w!=0)
fw0 = f0(wgrid[None], X, y).ravel()
minind = np.argmin(fw0);
minw = wgrid[minind]
plt.plot(wgrid, fw0, '.',markersize=7);
# plt.plot((minw, minw),plt.ylim(),'--k');
# plt.plot(minw, fw0[minind], 'r*',markersize=10);
plt.xlabel("$w$");
plt.ylabel("$f(w)$");
plt.title("L0 regularization"); # $f(w)=\sum_{i=1}^n (wx_i-y_i)^2$
print("minimizer: w=%.2f" % minw);

- L0-regularized minimum is often exactly at the ‘discontinuity’ at 0:
  - Sets the feature to exactly 0 (does feature selection), but is **non-convex**.
  
- We can fiddle with the code to see a case where we don't get $w=0$:
  - By making the un-regularized solution further from 0.
  - By decreasing $\lambda$.


#### L2 regularization

$$f(w)=\sum_{i=1}^n (wx_i-y_i)^2 + \lambda \|w\|^2$$

In [ ]:
def f2(w, X, y, λ=100):
    return np.sum((X@w-y)**2,axis=0) + λ*np.sum(w**2,axis=0)
fw2 = f2(wgrid[None], X, y).ravel()
minind = np.argmin(fw2);
minw = wgrid[minind]
plt.plot(wgrid, fw2);
plt.plot((minw, minw),plt.ylim(),'--k');
# plt.plot(minw, fw0[minind], 'r*',markersize=10);
plt.xlabel("$w$");
plt.ylabel("$f(w)$");
plt.title("L2 regularization"); # $f(w)=\sum_{i=1}^n (wx_i-y_i)^2$
print("minimizer: w=%.2f" % minw);

- Note the solution is closer to zero than without regularization, but still not zero.
- Why? Because no incentive to move all the way to zero; the slope of the regularization term goes to zero!

#### L1 regularization

$$f(w)=\sum_{i=1}^n (wx_i-y_i)^2 + \lambda \|w\|_1$$

In [ ]:
λ = 100
def f1(w, X, y, λ=100):
    return np.sum((X@w-y)**2,axis=0) + λ*np.sum(np.abs(w),axis=0)
fw1 = f1(wgrid[None], X, y).ravel()
minind = np.argmin(fw1);
minw = wgrid[minind]
plt.plot(wgrid, fw1);
plt.plot((minw, minw),plt.ylim(),'--k');
# plt.plot(minw, fw0[minind], 'r*',markersize=10);
plt.xlabel("$w$");
plt.ylabel("$f(w)$");
plt.title("L1 regularization"); # $f(w)=\sum_{i=1}^n (wx_i-y_i)^2$
print("minimizer: w=%.2f" % minw);

- Notice the solution is _exactly_ zero.
- Again, whether this happens depends on $\lambda$ and the original solution.

## Regularization strength intuition

Question: what is the effect of changing the regularization strength $\lambda$?

Let's take the oversimplified case of $n=1,d=1$ with no intercept and just one point at $(x_i,y_i)=(1,1)$. We can get zero training error by setting $w=1$:

In [ ]:
plt.plot(1,1,'.',markersize=20)
plt.plot((-3,3),(-3,3));
plt.xlabel("x");
plt.ylabel("y");
plt.title("Extreme toy example: $d=1,n=1$");

Our regularized loss is 

$$f(w)=(w-1)^2+\lambda w^2$$

- The left-hand term (data term) wants $w=1$
- The right-hand term (regularization term) wants $w=0$

In this particular case, because I picked such a simple example, we can do the math by hand. Taking the derivative and setting it to zero yields $\frac{df}{dw}=2(w-1)+2\lambda w=0$, so the minimum occurs at $w=\frac{1}{1+\lambda}$.

In [ ]:
x = np.linspace(0,100,1000)
plt.plot(x, 1/(1+x))
plt.xlabel('$\lambda$', fontsize=20)
plt.ylabel('optimal $w$', fontsize=20);

This is a _regularization path_ but in the very simple case of just one $w$. We recover the limiting cases:

- $\lambda=0$ gives us $w=1$
- $\lambda \rightarrow \infty$ gives us $w=0$

All this pretty much generalizes to larger $n$ and $d$. So $\lambda$ controls the relative importance of training error vs. regularization.

#### Regularization paths

Here's an example with $d>1,n>1$:

In [ ]:
d = 10
n = 30
w_true = npr.randn(d)*3
X = npr.randn(n,d)
y = npr.randn(n)/5 + X@w_true
λ_valsL2 = 10**np.linspace(-2,4,1000)
λ_valsL1 = 10**np.linspace(-2,2,1000)

WsL2 = np.zeros((len(λ_valsL2),d))
WsL1 = np.zeros((len(λ_valsL1),d))
for i,λ in enumerate(λ_valsL2):
    ridge = Ridge(alpha=λ)
    ridge.fit(X,y)
    WsL2[i] = ridge.coef_
for i,λ in enumerate(λ_valsL1):    
    lasso = Lasso(alpha=λ)
    lasso.fit(X,y)
    WsL1[i] = lasso.coef_

In [ ]:
plt.semilogx(λ_valsL2, WsL2);
plt.xlabel("λ");
plt.ylabel("$w_j$");
plt.title("L2 regularization");

In [ ]:
plt.semilogx(λ_valsL1, WsL1);
plt.xlabel("λ");
plt.ylabel("$w_j$");
plt.title("L1 regularization");

Notice how the weights "snap" to zero in the L1 case whereas they only approach 0 asymptotically in the L2 case.

(Bonus) Another question is what happens to regularization as $n$ gets large.

- As $n$ gets large, the loss term starts to dominate the regularization term for fixed $\lambda$, because it's a sum over $n$ things.
  - This appeals to our intuition, since we don't expect to overfit if we have lots of training data.
  - There's also a Bayesian interpretation coming later in the course.
- In theory, $\lambda$ should be $\Omega(1)$ and $O(n^{1/2})$ (hopefully those are the correct symbols!). 
  - If it grew linearly with $n$ then the loss and regularization would grow the same amount, which violates the above intuition about lots of training data.
- In practice, if we're choosing $\lambda$ with (cross)-validation anyway, we'll just get the value that works best.

## Why not use Huber regularization?

- Don’t confuse the L1 loss with L1-regularization!!!
  - L1-loss is robust to outlier data points.
    - You can use instead of removing outliers.
    - "sparse residuals"
  - L1-regularization is robust to irrelevant features.
    - You can use instead of removing features.
    - "sparse coefficients/weights"
- And note that you can use them together.
- Why aren’t we smoothing and using "Huber regularization"?
   - With the L1 loss, we cared about its behavior far from 0.
   - With L1 regularization, we care about its behavior near 0.
      - It’s precisely the non-smoothness that sets weights to exactly 0.

## Multiple choice question

Which of the following is true about L1 regularization? (There may be multiple that are true.)

1. For any value of $\lambda$, some of the $w$-values will be $0$.
2. For $\lambda \rightarrow \infty$ all of the $w$-values will be zero.
3. All the weights will be smaller than with L2 regularization, because you're not squaring them in the loss.
4. The regularization path is **not** a continuous function.


## Uniqueness of the solution: L1 and L2

- With no regularization, the solution is not unique. 
  - Example: two identical columns $i$ and $j$. 
    - If $(w_i,w_j)$ is a solution then $(w_j,w_i)$ is also a solution.
- With L1 regularization, the solution is also not unique. 
  - The same example applies.
- With L2 regularization, the solution is unique. 
  - If we have two identical columns $i$ and $j$ then $w_i=w_j$.
  
**The experiments shown below bonus material.**

In [ ]:
npr.seed(0)

n = 20
X = npr.randn(n,1)
y = npr.randn(n,1) + 0.5*X**2 + X
X = np.concatenate((X,X),axis=1) # identical columns

In [ ]:
gridX = np.linspace(-3,3,1000)
gridY = np.linspace(-3,3,1000)
meshX, meshY = np.meshgrid(gridX, gridY)

def make_plot(loss_fun, **kwargs):
    loss = loss_fun(np.c_[meshX.ravel(), meshY.ravel()].T, X, y, **kwargs)
    min_inds = np.nonzero(loss<=np.min(loss)*(1+1e-7))
    loss = loss.reshape(meshX.shape[0],meshX.shape[1])
    # plt.imshow(loss,extent=(np.min(gridX), np.max(gridX), np.min(gridY), np.max(gridY)), origin='lower')
    CS = plt.contourf(gridX, gridY, loss, 30);
    plt.plot(meshX.flatten()[min_inds], meshY.flatten()[min_inds], '.r');
    plt.axis('square');
    # plt.colorbar();
    plt.xlabel('$w_1$');
    plt.ylabel('$w_2$');
make_plot(f)
plt.title("No regularization");

There is a "smear" of solutions (red) along the $w_1+w_2=w_\text{total}$ axis for some $w_\text{total}$.

In [ ]:
lr = LinearRegression()
lr.fit(X,y)
print("w_total = %.4f"%np.sum(lr.coef_))

In [ ]:
make_plot(f1, λ=1)
plt.title("L1 regularization");

- This is very interesting! We still have $w_1+w_2=w_\text{total}$ but...
- we can't have one of them take the opposite sign of $w_\text{total}$ (in this case positive).

In [ ]:
make_plot(f2, λ=0.1)
plt.title("L2 regularization");

With L2, the solution is unique. Why? This isn't a general proof but, try minimizing $w_1^2+w_2^2$ subject to $w_1+w_2=w_\text{total}$. You'll get $w_1=w_2$.